In [8]:
import pandas as pd
import toml
import os
import yaml
from deepdiff import DeepDiff
from collections import defaultdict
import ast

In [9]:
def convert_value(value):
    """Chuyển đổi giá trị từ chuỗi sang kiểu dữ liệu thực tế (int, float, bool, None, list, dict)."""
    if pd.isna(value):
        return None  # Gán giá trị None thay vì chuỗi "null"
    
    if value in ["null"]:
        value = "null"
    elif value in ["True", "true"]:
        value = True
    elif value in ["False", "false"]:
        value = False
    
    try:
        return ast.literal_eval(str(value))  # Chuyển đổi nếu có thể
    except (ValueError, SyntaxError):
        return value  # Giữ nguyên nếu không thể chuyển đổi

def set_nested_value(d, keys, value):
    """Đặt giá trị vào từ điển lồng nhau và xử lý danh sách YAML đúng cách."""
    current_level = d
    for i, key in enumerate(keys[:-1]):
        if key.isdigit():  # Chuyển key số thành danh sách
            key = int(key)
        if isinstance(current_level, list):
            while len(current_level) <= key:
                current_level.append({})
            current_level = current_level[key]
        else:
            if key not in current_level:
                current_level[key] = [] if keys[i + 1].isdigit() else {}
            current_level = current_level[key]
    
    last_key = keys[-1]
    if last_key.isdigit():
        last_key = int(last_key)
    
    value = convert_value(value)  # Chuyển đổi giá trị

    if isinstance(current_level, list):
        current_level.append(value)
    elif isinstance(current_level, dict) and isinstance(last_key, int):
        if last_key >= len(current_level):
            current_level[last_key] = value
        else:
            current_level.append(value)
    else:
        current_level[last_key] = value

def excel_to_yaml(excel_file, yaml_file):
    df = pd.read_excel(excel_file, dtype=str)
    df = df.where(pd.notna(df), None)
    config = defaultdict(dict)
    
    for _, row in df.iterrows():
        keys = row['Parameter'].replace("[", ".").replace("]", "").split(".")  # Chuyển [0] thành .0
        # Xử lý đặc biệt cho metadata.annotations
        if keys[0] == "metadata" and keys[1] == "annotations":
            annotation_key = '.'.join(keys[2:])  # Giữ nguyên key annotations đầy đủ
            if "metadata" not in config:
                config["metadata"] = {}
            if "annotations" not in config["metadata"]:
                config["metadata"]["annotations"] = {}
            config["metadata"]["annotations"][annotation_key] = row['Value'] if pd.notna(row['Value']) else "null"
        else:
            set_nested_value(config, keys, row['Value'])

    with open(yaml_file, 'w') as file:
        yaml.dump(dict(config), file, default_flow_style=False, allow_unicode=True)
    
    print(f"YAML file has been generated: {yaml_file}")

In [10]:
def convert_none_values(obj):
    """Duyệt qua dictionary và chuyển đổi 'None' thành None (NoneType của Python)."""
    if isinstance(obj, dict):
        return {k: convert_none_values(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_none_values(v) for v in obj]
    elif obj == "None":  # Chuyển đổi chuỗi "None" thành None
        return None
    return obj

def load_yaml(file_path):
    """Đọc file YAML và trả về dưới dạng dictionary."""
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file) or {}
    return convert_none_values(data)

def compare_yaml_files(generated_yaml, actual_yaml, filename):
    """So sánh hai file YAML và in ra sự khác biệt."""
    generated_data = load_yaml(generated_yaml)
    actual_data = load_yaml(actual_yaml)
    
    diff = DeepDiff(actual_data, generated_data, ignore_order=True)
    
    if not diff:
        print(f"Hai file {filename} giống nhau.")
    else:
        print(f"Sự khác biệt giữa hai file {filename}:")
        print(diff)

In [14]:
# Sử dụng hàm với file Excel đầu vào và xuất ra file YAML
files = ["etcd", "kube-apiserver", "kube-scheduler", "kube-controller-manager", "audit-policy", "config", "crictl"]
# files = ["etcd"]
for f in files:
    excel_file = f"./parameterfiles/{f}.xlsx"
    yaml_file = f"./output/{f}.yaml"
    excel_to_yaml(excel_file, yaml_file)

YAML file has been generated: ./output/etcd.yaml
YAML file has been generated: ./output/kube-apiserver.yaml
YAML file has been generated: ./output/kube-scheduler.yaml
YAML file has been generated: ./output/kube-controller-manager.yaml
YAML file has been generated: ./output/audit-policy.yaml
YAML file has been generated: ./output/config.yaml
YAML file has been generated: ./output/crictl.yaml


In [15]:

files = ["etcd", "kube-apiserver", "kube-scheduler", "kube-controller-manager", "audit-policy", "config", "crictl"]
for f in files:
    # Đường dẫn file
    generated_yaml = f"./output/{f}.yaml"  # File YAML tạo từ Excel
    actual_yaml = f"./default/{f}.yaml"  # File YAML thực tế

    # Chạy so sánh
    compare_yaml_files(generated_yaml, actual_yaml, f)

Hai file etcd giống nhau.
Hai file kube-apiserver giống nhau.
Hai file kube-scheduler giống nhau.
Hai file kube-controller-manager giống nhau.
Hai file audit-policy giống nhau.
Hai file config giống nhau.
Hai file crictl giống nhau.


In [16]:
import toml
import pandas as pd
import ast
from deepdiff import DeepDiff
from collections import OrderedDict

### 📌 Bước 1: Chuyển file TOML thành file Excel ###
def flatten_dict(d, parent_key='', sep='.'):
    """Hàm làm phẳng dictionary lồng nhau thành key-value"""
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return OrderedDict(items)  # Duy trì thứ tự key

# Đọc file TOML
toml_file = "config.toml"
with open(toml_file, "r", encoding="utf-8") as file:
    toml_data = toml.load(file)

# Làm phẳng dictionary và lưu vào Excel
flat_data = flatten_dict(toml_data)
df = pd.DataFrame(flat_data.items(), columns=["Key", "Value"])
df.to_excel("config.xlsx", index=False)

print("✅ Đã chuyển đổi TOML thành Excel!")

### 📌 Bước 2: Chuyển file Excel về TOML ###
def nested_dict(keys, value):
    """Tạo dictionary lồng nhau từ danh sách keys"""
    d = value
    for key in reversed(keys):
        d = {key: d}
    return d

def merge_dict(d1, d2):
    """Hợp nhất 2 dictionary, đảm bảo không ghi đè dữ liệu"""
    for key, value in d2.items():
        if key in d1 and isinstance(d1[key], dict) and isinstance(value, dict):
            merge_dict(d1[key], value)
        else:
            d1[key] = value

def convert_value(value):
    """Chuyển đổi kiểu dữ liệu về đúng loại gốc"""
    if isinstance(value, str):
        value = value.strip()
        if value.lower() == "true":
            return True
        elif value.lower() == "false":
            return False
        elif value.isdigit():
            return int(value)
        else:
            try:
                return ast.literal_eval(value)  # Xử lý list, dict, float
            except (ValueError, SyntaxError):
                return value
    return value

# Đọc file Excel
df = pd.read_excel("config.xlsx")

# Chuyển Excel thành dictionary phân cấp
config_dict = OrderedDict()
for _, row in df.iterrows():
    keys = row["Key"].split(".")  # Tách key theo dấu chấm
    value = convert_value(row["Value"])
    merge_dict(config_dict, nested_dict(keys, value))

# Ghi file TOML mới
toml_new_file = "config_new.toml"
with open(toml_new_file, "w", encoding="utf-8") as file:
    toml.dump(config_dict, file)

print("✅ Đã chuyển đổi Excel thành TOML!")

### 📌 Bước 3: So sánh file TOML mới với file gốc ###
# Đọc lại 2 file TOML để so sánh
with open("config.toml", "r", encoding="utf-8") as file:
    toml_original = toml.load(file)

with open("config_new.toml", "r", encoding="utf-8") as file:
    toml_new = toml.load(file)

# So sánh hai file
diff = DeepDiff(toml_original, toml_new, ignore_order=True)

# In ra sự khác biệt
if diff:
    print("⚠️ Có sự khác biệt giữa config.toml và config_new.toml:")
    print(diff)
else:
    print("✅ Hai file TOML giống nhau hoàn toàn!")


✅ Đã chuyển đổi TOML thành Excel!
✅ Đã chuyển đổi Excel thành TOML!
⚠️ Có sự khác biệt giữa config.toml và config_new.toml:
{'type_changes': {"root['plugin_dir']": {'old_type': <class 'str'>, 'new_type': <class 'float'>, 'old_value': '', 'new_value': nan}, "root['temp']": {'old_type': <class 'str'>, 'new_type': <class 'float'>, 'old_value': '', 'new_value': nan}, "root['cgroup']['path']": {'old_type': <class 'str'>, 'new_type': <class 'float'>, 'old_value': '', 'new_value': nan}, "root['debug']['address']": {'old_type': <class 'str'>, 'new_type': <class 'float'>, 'old_value': '', 'new_value': nan}, "root['debug']['format']": {'old_type': <class 'str'>, 'new_type': <class 'float'>, 'old_value': '', 'new_value': nan}, "root['debug']['level']": {'old_type': <class 'str'>, 'new_type': <class 'float'>, 'old_value': '', 'new_value': nan}, "root['grpc']['tcp_address']": {'old_type': <class 'str'>, 'new_type': <class 'float'>, 'old_value': '', 'new_value': nan}, "root['grpc']['tcp_tls_ca']": {

In [17]:
import toml
import pandas as pd
import json
from deepdiff import DeepDiff

### 📌 Bước 1: Chuyển TOML thành Excel ###
def save_toml_to_excel(toml_path, excel_path):
    """Chuyển đổi TOML thành file Excel mà vẫn giữ nguyên cấu trúc"""
    with open(toml_path, "r", encoding="utf-8") as f:
        data = toml.load(f)

    # Convert dữ liệu thành JSON string để giữ nguyên cấu trúc
    df = pd.DataFrame({"Key": list(data.keys()), "Value": [json.dumps(v) for v in data.values()]})
    
    # Lưu ra file Excel
    df.to_excel(excel_path, index=False)
    print("✅ Đã chuyển đổi TOML thành Excel!")

### 📌 Bước 2: Chuyển Excel về TOML ###
def save_excel_to_toml(excel_path, toml_path):
    """Đọc Excel và khôi phục lại TOML y hệt file gốc"""
    df = pd.read_excel(excel_path, dtype=str)

    # Convert từ JSON string về dictionary gốc
    toml_data = {row["Key"]: json.loads(row["Value"]) for _, row in df.iterrows()}

    # Lưu file TOML mới
    with open(toml_path, "w", encoding="utf-8") as f:
        toml.dump(toml_data, f)

    print("✅ Đã chuyển đổi Excel thành TOML!")

### 📌 Bước 3: Kiểm tra sự khác biệt ###
def compare_toml_files(original_toml, new_toml):
    """So sánh hai file TOML để đảm bảo không có sự khác biệt"""
    with open(original_toml, "r", encoding="utf-8") as f:
        data1 = toml.load(f)
    
    with open(new_toml, "r", encoding="utf-8") as f:
        data2 = toml.load(f)

    diff = DeepDiff(data1, data2, ignore_order=True)
    
    if diff:
        print("⚠️ Có sự khác biệt giữa config.toml và config_new.toml:")
        print(diff)
    else:
        print("✅ Hai file TOML giống nhau hoàn toàn!")

### 📌 Chạy quy trình ###
toml_file = "config.toml"
excel_file = "config.xlsx"
new_toml_file = "config_new.toml"

save_toml_to_excel(toml_file, excel_file)  # Chuyển TOML thành Excel
save_excel_to_toml(excel_file, new_toml_file)  # Chuyển Excel về TOML
compare_toml_files(toml_file, new_toml_file)  # So sánh hai file


✅ Đã chuyển đổi TOML thành Excel!
✅ Đã chuyển đổi Excel thành TOML!
✅ Hai file TOML giống nhau hoàn toàn!


In [8]:
import toml
import pandas as pd
from deepdiff import DeepDiff
import ast
import numpy as np

# Đọc file TOML gốc
with open("config.toml", "r") as f:
    config = toml.load(f)

# Chuyển đổi kiểu dữ liệu an toàn
def safe_convert(value):
    if isinstance(value, (list, dict, bool)):
        return str(value)  # Giữ nguyên kiểu khi lưu Excel
    return value

# Chuyển TOML thành DataFrame
def flatten_dict(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, safe_convert(v)))
    return dict(items)

flat_config = flatten_dict(config)
df = pd.DataFrame(list(flat_config.items()), columns=["Key", "Value"])
df.to_excel("config.xlsx", index=False)

# Đọc lại từ Excel
df = pd.read_excel("config.xlsx")

new_config = {}
for index, row in df.iterrows():
    keys = row["Key"].split(".")
    value = row["Value"]

    # Xử lý kiểu dữ liệu
    if isinstance(value, str):
        if value.strip() in ["nan", "None"]:  # Nếu là NaN hoặc None, đổi về ""
            value = ""
        elif value in ["True", "False"]:  # Giữ nguyên kiểu `bool`
            value = value == "True"
        elif value.startswith("[") and value.endswith("]"):  # Chuyển lại thành `list`
            try:
                value = ast.literal_eval(value)
            except:
                pass
        elif value.startswith("{") and value.endswith("}"):  # Chuyển lại thành `dict`
            try:
                value = ast.literal_eval(value)
            except:
                pass

    elif isinstance(value, float) and np.isnan(value):
        value = ""  # Đổi NaN về ""

    d = new_config
    for k in keys[:-1]:
        d = d.setdefault(k, {})
    d[keys[-1]] = value

# Đảm bảo không mất `proxy_plugins`
if "proxy_plugins" not in new_config:
    new_config["proxy_plugins"] = config.get("proxy_plugins", {})

# So sánh với bản gốc
diff = DeepDiff(config, new_config, ignore_order=True, significant_digits=3)

# Lưu lại file TOML mới
with open("config_new.toml", "w") as f:
    toml.dump(new_config, f)

# Kết quả
print("✅ Đã lưu", len(flat_config), "tham số vào file Excel!")
print("✅ Đã khôi phục", len(flat_config), "tham số vào file TOML!")
if diff:
    print("⚠️ Có sự khác biệt giữa config.toml và config_new.toml:")
    print(diff)
else:
    print("✅ Cấu trúc không thay đổi! 🎉")


✅ Đã lưu 194 tham số vào file Excel!
✅ Đã khôi phục 194 tham số vào file TOML!
⚠️ Có sự khác biệt giữa config.toml và config_new.toml:
{'values_changed': {"root['plugins']": {'new_value': {'io': {'containerd': {'gc': {'v1': {'scheduler': {'deletion_threshold': 0, 'mutation_threshold': 100, 'pause_threshold': 0.02, 'schedule_delay': '0s', 'startup_delay': '100ms'}}}, 'grpc': {'v1': {'cri': {'cdi_spec_dirs': ['/etc/cdi', '/var/run/cdi'], 'device_ownership_from_security_context': False, 'disable_apparmor': False, 'disable_cgroup': False, 'disable_hugetlb_controller': True, 'disable_proc_mount': False, 'disable_tcp_service': True, 'drain_exec_sync_io_timeout': '0s', 'enable_cdi': False, 'enable_selinux': False, 'enable_tls_streaming': False, 'enable_unprivileged_icmp': False, 'enable_unprivileged_ports': False, 'ignore_image_defined_volumes': False, 'image_pull_progress_timeout': '5m0s', 'max_concurrent_downloads': 3, 'max_container_log_line_size': 16384, 'netns_mounts_under_state_dir': Fa